In [19]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
from mlxtend.frequent_patterns import association_rules

In [20]:
data = pd.read_csv('../data/data_msg_only(new tags).csv')

In [21]:
def custom_round(x, base=10):
    return '_' + str(int(base * round(float(x)/base)))

In [22]:
def map_and_or_count(x):
    if x == 0:
        return '0_and_or'
    if x == 1:
        return '1_and_or'

In [23]:
def map_blank_line(x):
    if x == 0:
        return '0_blank_line'
    if x == 1:
        return '1_blank_line'

In [24]:
def map_capital_start(x):
    if x == 0:
        return '0_capital_start'
    if x == 1:
        return '1_capital_start'

In [25]:
def map_end_dot(x):
    if x == 0:
        return '0_end_dot'
    if x == 1:
        return '1_end_dot'

In [26]:
def map_imperative_start(x):
    if x == 0:
        return '0_imperative_start'
    if x == 1:
        return '1_imperative_start'

In [27]:
def map_wrap_to_72(x):
    if x == 0:
        return '0_wrap_to_72'
    if x == 1:
        return '1_wrap_to_72'

In [28]:
def map_verb_obj_conn(x):
    if x == 0:
        return '0_verb_obj_conn'
    if x == 1:
        return '1_verb_obj_conn'

In [29]:
data['characters_count'] = pd.Series(data['characters_count']).apply(lambda x: custom_round(x, base=5))
data['and_or_count'] = pd.Series(data['and_or']).apply(lambda x: map_and_or_count(x))
data['blank_line'] = pd.Series(data['blank_line']).apply(lambda x: map_blank_line(x))
data['capital_start'] = pd.Series(data['capital_start']).apply(lambda x: map_capital_start(x))
data['end_dot'] = pd.Series(data['end_dot']).apply(lambda x: map_end_dot(x))
data['imperative_start'] = pd.Series(data['imperative_start']).apply(lambda x: map_imperative_start(x))
data['wrap_to_72'] = pd.Series(data['wrap_to_72']).apply(lambda x: map_wrap_to_72(x))
data['verb_obj_conn'] = pd.Series(data['verb_obj_conn']).apply(lambda x: map_verb_obj_conn(x))

In [30]:
data.head()

,characters_count,and_or_count,blank_line,capital_start,end_dot,imperative_start,wrap_to_72,verb_obj_conn,label
0,_30,1_and_or_count,1_blank_line,1_capital_start,1_end_dot,1_imperative_start,1_wrap_to_72,1_verb_obj_conn,good
1,_35,1_and_or_count,1_blank_line,1_capital_start,1_end_dot,1_imperative_start,1_wrap_to_72,1_verb_obj_conn,good
2,_45,1_and_or_count,1_blank_line,1_capital_start,0_end_dot,1_imperative_start,1_wrap_to_72,1_verb_obj_conn,good
3,_40,1_and_or_count,1_blank_line,1_capital_start,1_end_dot,1_imperative_start,1_wrap_to_72,1_verb_obj_conn,good
4,_30,1_and_or_count,1_blank_line,1_capital_start,1_end_dot,1_imperative_start,1_wrap_to_72,1_verb_obj_conn,good


In [31]:
dataset  = data.values.tolist()

In [32]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [33]:
frequent_itemsets = fpgrowth(df, min_support=0.2, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.975,(1_blank_line)
1,0.880,(1_and_or_count)
2,0.793,(1_verb_obj_conn)
3,0.791,(1_end_dot)
4,0.783,(1_wrap_to_72)
...,...,...
314,0.230,"(1_wrap_to_72, 0_imperative_start, 1_and_or_co..."
315,0.236,"(1_wrap_to_72, 1_blank_line, 0_imperative_star..."
316,0.229,"(1_wrap_to_72, 1_blank_line, 0_imperative_star..."
317,0.218,"(1_wrap_to_72, 1_end_dot, 0_imperative_start, ..."


In [34]:
frequent_itemsets = apriori(df, min_support=0.2, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.311,(0_capital_start)
1,0.209,(0_end_dot)
2,0.385,(0_imperative_start)
3,0.207,(0_verb_obj_conn)
4,0.217,(0_wrap_to_72)
...,...,...
314,0.259,"(1_wrap_to_72, 1_capital_start, 1_and_or_count..."
315,0.259,"(1_wrap_to_72, 1_end_dot, 1_and_or_count, 1_im..."
316,0.218,"(1_wrap_to_72, 1_capital_start, 1_and_or_count..."
317,0.229,"(1_wrap_to_72, 1_capital_start, 1_end_dot, 1_i..."


In [35]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.8)

rules_good = rules[rules['consequents'].astype(str).str.contains('{\'good\'}')]
rules_neutral = rules[rules['consequents'].astype(str).str.contains('{\'neutral\'}')]
rules_bad = rules[rules['consequents'].astype(str).str.contains('{\'bad\'}')]

# rules_good.to_csv('adsf.csv')
rules_good
# rules_neutral
# rules_bad
# rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
766,"(1_end_dot, 1_verb_obj_conn, 1_and_or_count, 1...",(good),0.336,0.413,0.275,0.818452,1.981725,0.136232,3.233311
1094,"(1_and_or_count, 1_imperative_start, 1_blank_l...",(good),0.336,0.413,0.275,0.818452,1.981725,0.136232,3.233311
1180,"(1_capital_start, 1_and_or_count, 1_imperative...",(good),0.276,0.413,0.230,0.833333,2.017756,0.116012,3.522000
1204,"(1_wrap_to_72, 1_capital_start, 1_and_or_count...",(good),0.295,0.413,0.236,0.800000,1.937046,0.114165,2.935000
1233,"(1_wrap_to_72, 1_and_or_count, 1_imperative_st...",(good),0.308,0.413,0.259,0.840909,2.036099,0.131796,3.689714
1333,"(1_wrap_to_72, 1_capital_start, 1_imperative_s...",(good),0.282,0.413,0.229,0.812057,1.966239,0.112534,3.123283
1362,"(1_capital_start, 1_end_dot, 1_and_or_count, 1...",(good),0.276,0.413,0.230,0.833333,2.017756,0.116012,3.522000
1412,"(1_wrap_to_72, 1_capital_start, 1_and_or_count...",(good),0.295,0.413,0.236,0.800000,1.937046,0.114165,2.935000
1473,"(1_wrap_to_72, 1_and_or_count, 1_imperative_st...",(good),0.308,0.413,0.259,0.840909,2.036099,0.131796,3.689714
1506,"(1_wrap_to_72, 1_capital_start, 1_and_or_count...",(good),0.253,0.413,0.218,0.861660,2.086344,0.113511,4.243171


In [36]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.8)

rules_good = rules[rules['consequents'].astype(str).str.contains('{\'good\'}')]
rules_neutral = rules[rules['consequents'].astype(str).str.contains('{\'neutral\'}')]
rules_bad = rules[rules['consequents'].astype(str).str.contains('{\'bad\'}')]

# rules_good.to_csv('adsf.csv')
rules_good
# rules_neutral
# rules_bad
# rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
766,"(1_end_dot, 1_verb_obj_conn, 1_and_or_count, 1...",(good),0.336,0.413,0.275,0.818452,1.981725,0.136232,3.233311
1094,"(1_and_or_count, 1_imperative_start, 1_blank_l...",(good),0.336,0.413,0.275,0.818452,1.981725,0.136232,3.233311
1180,"(1_capital_start, 1_and_or_count, 1_imperative...",(good),0.276,0.413,0.230,0.833333,2.017756,0.116012,3.522000
1204,"(1_wrap_to_72, 1_capital_start, 1_and_or_count...",(good),0.295,0.413,0.236,0.800000,1.937046,0.114165,2.935000
1233,"(1_wrap_to_72, 1_and_or_count, 1_imperative_st...",(good),0.308,0.413,0.259,0.840909,2.036099,0.131796,3.689714
1333,"(1_wrap_to_72, 1_capital_start, 1_imperative_s...",(good),0.282,0.413,0.229,0.812057,1.966239,0.112534,3.123283
1362,"(1_capital_start, 1_end_dot, 1_and_or_count, 1...",(good),0.276,0.413,0.230,0.833333,2.017756,0.116012,3.522000
1412,"(1_wrap_to_72, 1_capital_start, 1_and_or_count...",(good),0.295,0.413,0.236,0.800000,1.937046,0.114165,2.935000
1473,"(1_wrap_to_72, 1_and_or_count, 1_imperative_st...",(good),0.308,0.413,0.259,0.840909,2.036099,0.131796,3.689714
1506,"(1_wrap_to_72, 1_capital_start, 1_and_or_count...",(good),0.253,0.413,0.218,0.861660,2.086344,0.113511,4.243171
